# Feature Engineering Testing

In [24]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [25]:
red_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
white_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"

red_df = pd.read_csv(red_url, sep=";")
white_df = pd.read_csv(white_url, sep=";")

In [26]:
red_df['is_red'] = 1
white_df['is_red'] = 0
df_raw = pd.concat([red_df, white_df])
df_raw.columns = [x.lower().replace(" ", "_") for x in df_raw.columns]

In [27]:
df_raw

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality,is_red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [32]:
df_np = df_raw.to_numpy()
y = df_np[:, -2]
X = np.delete(X, -2, axis=1)

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
xgboost = xgb.XGBClassifier()


0.4661364802462801

In [51]:
from sklearn.metrics import top_k_accuracy_score
cross_val_score(xgboost, X_train, y_train, cv = 10, scoring = "accuracy")




/Users/josephdespres/my_env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(


array([0.60512821, 0.56923077, 0.57179487, 0.53589744, 0.59230769,
       0.55897436, 0.56153846, 0.55128205, 0.6066838 , 0.63239075])

In [49]:
df_raw = df_raw.jgroupby(df_raw.columns.tolist(), as_index=False).size()
df_raw.rename(columns = {'size':'duplicates'}, inplace = True)

AttributeError: 'DataFrame' object has no attribute 'jgroupby'

In [ ]:
y = df_raw.quality
X = df_raw.drop("quality", axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)

In [ ]:
def save_train_test_arrays(n):
    np.savez(f"../data/processed/train_test_split{n}.npz", 
                X_train = X_train,
                X_test = X_test,
                y_train = y_train,
                y_test = y_test)
    return n+1

In [ ]:
n = 0
n = save_train_test_arrays(n)

In [ ]:
from sklearn import preprocessing
already_scaled = X_train[["is_red", "duplicates"]].to_numpy()
to_scale = X_train.drop(["is_red", "duplicates"], axis=1)
scaler = preprocessing.StandardScaler().fit(to_scale)
scaled = scaler.transform(to_scale)
n = save_train_test_arrays(n)

In [ ]:
upper_quartile = np.quantile(scaled, 0.75, axis=0) * 3
lower_quartile = np.quantile(scaled, 0.25, axis=0) * 3
scaled = np.where(scaled > upper_quartile, np.nan, scaled)
scaled = np.where(scaled < lower_quartile, np.nan, scaled)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
inputed = imputer.fit_transform(scaled)
X_train = np.concatenate((inputed, already_scaled), axis=1)

In [ ]:
# im gona regret not putting this in a function
test_scaled = X_test[["is_red", "duplicates"]].to_numpy()
test_to_scale = X_test.drop(["is_red", "duplicates"], axis=1)
test_scaled = scaler.transform(test_to_scale)
test_scaled = np.where(test_scaled > upper_quartile, np.nan, test_scaled)
test_scaled = np.where(test_scaled < lower_quartile, np.nan, test_scaled)
test_inputed = imputer.fit_transform(test_scaled)
X_test = np.concatenate((test_inputed, test_scaled), axis=1)

In [ ]:
n = save_train_test_arrays(n)